# Activation Grids
- This notebook does not do any visualizations, it just creates the act grids for each layer and saves them

In [2]:
import numpy as np
import tensorflow as tf


from sklearn.preprocessing import normalize
import numpy as np

import matplotlib
import matplotlib.pyplot as plt

from PIL import Image
import math
import requests
from io import BytesIO


import lucid.modelzoo.vision_models as models
import lucid.optvis.render as render
from lucid.misc.io import show, load
from lucid.misc.io.showing import _image_url
import lucid.scratch.web.svelte as lucid_svelte

import lucid.modelzoo.vision_models as models
from lucid.misc.io import show
import lucid.optvis.objectives as objectives
import lucid.optvis.param as param
import lucid.optvis.render as render
import lucid.optvis.transform as transform

In [5]:
LAYERS = { 'conv1': 'Conv2D',
           'conv1_relu':'Relu',
           'pool1': 'MaxPool',
           'conv2': 'Conv2D_1',
           'conv2_relu': 'Relu_1',
           'pool2': 'MaxPool_1',
           'conv3': 'Conv2D_2',
           'conv3_relu': 'Relu_2',
           'pool3': 'MaxPool_2',
         }


In [6]:
from lucid.modelzoo.vision_base import Model

class DLA(Model):
    model_path = 'https://storage.googleapis.com/dla_protobuff/full_model_8_13.pb'
    image_shape = [1, 400]
    image_value_range = [0, 1]
    input_name = 'x'

In [7]:
def resize_input(image_file):
    img = Image.open(image_file)
    img = img.resize((400,267), Image.ANTIALIAS)
    img.save(image_file)
    
def run(data, start, layer, hide=False):
    flux = np.load(data)
    flux_test = flux[start:start+400]
    flux_test = flux_test.reshape(1, 400, 1, 1)
    flux_graph = flux_test.reshape(400)
    fig = plt.figure(frameon=False);
    ax = plt.Axes(fig, [0, 0, 1, 1]);
    ax.set_axis_off();
    fig.add_axes(ax);
    ax.plot(flux_graph, 'black');
    ax.set(xlim=(0, 400));
    file_save = 'flux_' + str(start) + '.png'
    fig.savefig(file_save);
    plt.close(fig)
    resize_input(file_save)
    acts = render_act_grid(layer, flux_test, file_save)
    return acts

In [8]:
model = DLA()

def render_act_grid(layer, test_input):
    input_1d = test_input # Actual 1-Dimensional test input
    
    model_layer = LAYERS[layer]
    
    # Compute the activations
    with tf.Graph().as_default(), tf.Session() as sess:
        t_input = tf.placeholder(tf.float32, shape=[1, 400, 1, 1])
        T = render.import_model(model, t_input, t_input)
        acts = T(model_layer).eval(feed_dict={t_input: input_1d, 'import/keep_prob:0': .98})[0]
    acts_flat = acts.reshape([-1] + [acts.shape[2]])
    
    # Render an image for each activation vector
    param_f = lambda: param.image(400, h=1, channels=1, batch = acts.shape[0])
    obj = objectives.Objective.sum(
        [objectives.direction(model_layer, v, batch=n)
         for n,v in enumerate(acts_flat)])
    
    thresholds = (128,)
    vis_imgs = render.render_vis(model, obj, param_f, thresholds=thresholds, transforms=[], verbose=False)[-1]
    vis_imgs = vis_imgs.reshape(acts.shape[0], 400)
    
    return vis_imgs
        

In [13]:
# Specify the 400 length flux input
flux = np.load("../data/spectra/flux0.npy")
flux = flux[0:400]
flux = flux.reshape(1, 400, 1, 1)
file_save = 'flux0_'

In [27]:
# Specify the 400 length flux input
flux = np.load("../data/spectra/flux.npy")
flux = flux[700:1100]
flux = flux.reshape(1, 400, 1, 1)
file_save = 'flux700_'

In [28]:
for layer in LAYERS:
    print(layer + "...")
    imgs = render_act_grid(layer, flux)
    img_flat = imgs.flatten()
    out = file_save + layer + ".npy"
    np.save(out, img_flat)

conv1...
conv1_relu...
pool1...
conv2...
conv2_relu...
pool2...
conv3...
conv3_relu...
pool3...


In [29]:
# Specify the 400 length flux input
flux = np.load("../data/spectra/flux.npy")
flux = flux[350:750]
flux = flux.reshape(1, 400, 1, 1)
file_save = 'flux350_'

In [30]:
for layer in LAYERS:
    print(layer + "...")
    imgs = render_act_grid(layer, flux)
    img_flat = imgs.flatten()
    out = file_save + layer + ".npy"
    np.save(out, img_flat)

conv1...
conv1_relu...
pool1...
conv2...
conv2_relu...
pool2...
conv3...
conv3_relu...
pool3...


In [18]:
# Specify the 400 length flux input
flux = np.load("../data/spectra/flux1.npy")
flux = flux[1000:1400]
flux = flux.reshape(1, 400, 1, 1)
file_save = 'flux1_'

In [19]:
for layer in LAYERS:
    print(layer + "...")
    imgs = render_act_grid(layer, flux)
    img_flat = imgs.flatten()
    out = file_save + layer + ".npy"
    np.save(out, img_flat)

conv1...
conv1_relu...
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


pool1...
conv2...
conv2_relu...
pool2...
conv3...
conv3_relu...
pool3...
